01 키(Height), 몸무게(Weight), 발 사이즈(Shoe Size)를 이용하여 남녀(Sex) 성별을 분류하는 로지스틱 회귀모형을 만들고자 한다. 주어진 데이터 세트를 이용하여 로지스틱 회귀모형을 만들고 아래의 질문에 답하시오. 단, 주어진 데이터 세트의 2/3를 훈련용으로, 1/3을 테스트용으로 분할하여 작업하시오.

(1) 훈련용 데이터를 이용하여 'Height', 'Weight', 'ShoeSize'를 설명변수로 사용하는 로지스틱 회귀모형을 만들고 Weight의 오즈비를 계산하시오.

(2) 모델의 로짓우도(log-likelihood)를 구하시오.

(3) (2)에서 만든 모델로 평가용 데이터를 예측한 결과와 실제값의 오차율을 구하시오.

In [21]:
# 필요한 라이브러리 가져오기
import pandas as pd
# 문제에서 요구하는 로지스틱 회귀모형을 만들기 위해서 statsmodel.api를 사용하면 통계분석 모델을
# 쉽게 만들 수 있다.
import statsmodels.api as sm
file = '/content/07.03.01-gender_prediction_dataset.csv'
df = pd.read_csv(file)
# print(df)

# 로지스틱 회귀모형을 만들기 위해 Gender를 이진 변수로 인코딩을 하자. 인코딩 컬럼이 하나뿐이고
# 카테고리도 두 개 뿐이기 때문에 간단히 map 함수를 이용해서 인코딩을 하도록 하자
df1 = df.copy()
# print(df.info())

# Gender를 이진 변수로 변환하기
df1['Gender'] = df1['Gender'].map({'Male' : 1, 'Female' : 0})
# print(df1.head())
# print('*' * 100)
# 데이터프레임의 unique()는 데이터에 고유값들이 어떠한 종류들이 있는지 알고 싶을 때 사용하는 함수
# print(df1['Gender'].unique())

# 모델을 만들고 오즈비를 계산해보자.
# 오즈비(Odds Ratio, 교차비, 승산비)
# 오즈비는 특정 사건이 발생할 확률(p)과 그 사건이 발생하지 않을 확률(1-p)의 비를 의미한다.
# 사건이 발생할 확률(p) / 사건이 발생하지 않을 확률(p-1)
# 예를 들어 어떤 x 변수의 오즈비가 1.5일 경우 해당 변수의 값이 1만큼 늘어나면 y값에 1.5만큼
# 영향을 준다고 할 수 있다.
# 문제에서 데이터를 분리하여 진행할 것을 주문했다. 따라서 sklearn의 model_selection 패키지를
# 이용하여 데이터를 분리할 것이다. 이때 카테고리 분류 모델이기 때문에 stratify파라미터를 적용한다.
# 변수 설정하기
X = df1[['Height', 'Weight','ShoeSize']]
y = df1['Gender']

# 데이터 분리하기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size = 0.3,
                                                    stratify=y.values.ravel())
# print(len(X_train), len(X_test))

# 로지스틱 회귀 모형 생성 및 훈련
X_train = sm.add_constant(X_train)  # 절편 추가
lm = sm.Logit(y_train, X_train.values).fit()  # X_train을 ndarray로 입력

# 모델 요약 출력
# print(lm.summary())

# 모델 요약정보에서 x2가 Weight 변수에 해당한다. 오즈비는 앞서 설명했듯이 성공할 확률과
# 실패할 확률의 비를 의미한다. 모델의 회귀계수를 지수로 변환하면 오즈비가 된다.
# 지수 변환을 위해서 넘파이의 exp 함수를 사용한다.
# Weight의 odds 비 구하기
# 오즈비는 X변수의 값이 1만큼 늘어나면 y값에 오즈비(ex.1.5배) 만큼 영향을 준다는 것이다.
# np.exp 는 밑수인 a가 자연 상수 'e'(약, 2.71828)이고, 이에 대한 입력값 n제곱을 한 것을 의미한다.
# 지수를 얻기 한 것으로 회귀계수(coef)를 지수로 변환하면 바로 오즈비가 된다.
import numpy as np
odds_ratios = np.exp(lm.params)
# print(odds_ratios)
result = odds_ratios[2] # Weight는 두 번째 인덱스
print(result)  # 작은 문제 1번의 답


# 작은 문제 2번째
# 모델의 로그우도(log-likelihood)를 구하시오.
# 로그우도는 통계 모델링에서 중요한 개념이다. 관측된 데이터가 특정 통계 모델에
# 얼마나 잘 맞는지를 나타내는 지표다. 로그우도는 우도에 로그를 취한 값을 말하며,
# 우도는 주어진 모델의 매개변수가 특정 값일 때 관측된 데이터가 나타날 확률을 의미
# 한다. 다시 말해 모델이 주어진 데이터를 생성할 확률로, 모델의 데이터 적합성을
# 볼 때 사용할 수 있다. 특정 모델이 주어진 데이터를 생성할 확률이 높다면 모델이
# 데이터에 잘 맞는 것으로 판단할 수가 있다. 그런데 우도 값은 확률의 곱으로 표현되므로
# 아주 작아질 수 있다. 이런 불편을 해소하기 위해서 로그를 취한다. 로그우도는 우도에
# 로그 값을 취한 것이다. 따라서 우도 값이 높을수록 로그우도 값도 높아진다.
# 머신러닝에서는 로그우도의 값을 최대화하는 매개변수(하이퍼 파라미터)값을 찾는 것을
# 목표로 한다. 다시 말해 로그우도는 모델이 데이터에 얼마나 잘 맞는지를 나타내는 척도로
# 이용이 되는 지표다.
# 로그우도는 모델에서 llf를 변수를 이용해 가져올 수가 있다. 로그우도 값은 음수이며 음수의
# 절대값이 작으면 작을수록 모델 적합도가 높다는 것을 나타낸다. 따라서 로그우도 값을
# 이용해서 여러 개 모델의 적합도를 비교하는 데 사용할 수 있다.
# 로그우도 구하기
log_likelihood = lm.llf
print(log_likelihood)

# 작은 문제 3번
# (3) (2)에서 만든 모델로 평가용 데이터를 예측한 결과와 실제값의 오차율을 구하시오.
# 앞서 분리해둔 테스트 데이터를 이용해서 예측값과 실제값의 오차율을 구해보자.
# 기존 훈련용 데이터로 생성한 모델에 테스트 데이터를 입력하여 1, 0으로 표현하는
# 예측 데이터를 만들자.
# 평가용 데이터로 예측하기
X_test = sm.add_constant(X_test)
y_pred = (lm.predict(X_test) >= 0.5).astype(int)
# print(y_pred.values)
# print('*' * 100)
# print(y_test)

# error율 구해서 출력하기
error_rate = round(np.mean(np.abs(y_pred.values - y_test)), 4)
print(error_rate)


Optimization terminated successfully.
         Current function value: 0.058667
         Iterations 11
1.42998768731743
-12.320131393764626
0.0222
